In [1]:
import os
import sys
import re
import time
from github import Github
from random import randint
from pprint import pprint

# Credentials

In [2]:
from pymongo import MongoClient
pw_file = 'credentials/pw.txt'
if os.path.exists(pw_file): 
    with open(pw_file, 'r') as f:
        email, indeed_pw = f.readline().strip().split(', ')
        username, pia_pw = f.readline().strip().split(', ')
        pub_ip, mongo_usr, mongo_usr_pw = f.readline().strip().split(', ')

# Connect to MongoDB

In [3]:
# connect to ec2 mongo client
client = MongoClient('{0}:27017'.format(pub_ip))

# get reference to  resume_db
db = client['github_db']

# authenticate user for database
db.authenticate(mongo_usr, mongo_usr_pw)

True

# Create a MongoDB collection

In [397]:
for name in db.collection_names():
    #db.drop_collection(name)
    print('Number of documents in \"{0}\" collection: {1}'.format(name, db[name].count()))

In [398]:
# create a collection in the 'github_db' database
lst_collections = ['git_users', 'git_users_following', 'git_users_followers', 'git_users_starred', 
                   'git_repos_meta', 'git_repos_docs', 'git_repos_subscribers', 'git_repos_stargazers', 
                  'git_repos_contributors']

for collection in lst_collections:
    if collection not in db.collection_names():
        db.create_collection(collection)

In [399]:
for name in db.collection_names():
    print('Number of documents in \"{0}\" collection: {1}'.format(name, db[name].count()))

Number of documents in "git_repos_stargazers" collection: 0
Number of documents in "git_users_starred" collection: 0
Number of documents in "git_repos_meta" collection: 0
Number of documents in "git_users_followers" collection: 0
Number of documents in "git_repos_contributors" collection: 0
Number of documents in "git_repos_subscribers" collection: 0
Number of documents in "git_users_following" collection: 0
Number of documents in "git_repos_docs" collection: 0
Number of documents in "git_users" collection: 0


# Get authenticated access to Github for higher requests limit

In [352]:
# import github private token
with open('credentials/token.txt', 'r') as infile:
    token = infile.readline().strip()
    user = infile.readline().strip()

git_client = Github(token)

# Rate limits

In [353]:
def get_remaining(_type ='core'):
    '''
    _type = 'core' or 'search'
    Return: tuple of remaining rate limit quantity, and time till reset
    '''
    
    rate_limit = git_client.get_rate_limit()
    raw = dict(rate_limit.raw_data)
    
    remaining = int(raw['resources'][_type]['remaining'])
    reset = int(raw['resources'][_type]['reset'])
    
    return (remaining, reset)

In [354]:
print('Resource rate limit remaining: {0}'.format(get_remaining('core')))
print('Search rate limit remaining: {0}'.format(get_remaining('search')))

Resource rate limit remaining: (5000, 1472842910)
Search rate limit remaining: (30, 1472839370)


# REMOVE ME - ONLY FOR DEV

In [355]:
USER = 'ptwobrussell'
REPO = 'Mining-the-Social-Web'

# check for rate limit
limit_wait()

user = git_client.get_user(USER)
repo = user.get_repo(REPO)

# Repo Metadata Functions

In [356]:
def repo_subscribers(repo):
    '''
    IN: github repo object
    RETURN: dictionary of subscribers keyed by repo full_name
    '''
    repo_dict = {}
    try:
        # check for rate limit
        limit_wait()
        
        repo_dict[repo.full_name] = [f.login for f in repo.get_subscribers()]
    except:
        repo_dict[repo.full_name] = ''
        
    return repo_dict

In [357]:
def repo_stargazers(repo):
    '''
    IN: github repo object
    RETURN: dictionary of stargazers keyed by repo full_name
    '''
    repo_dict = {}
    try:
        # check for rate limit
        limit_wait()
    
        repo_dict[repo.full_name] = [f.login for f in repo.get_stargazers()]
    except:
        repo_dict[repo.full_name] = ''

    return repo_dict

In [358]:
def repo_contributors(repo):
    '''
    IN: github repo object
    RETURN: dictionary of contributors keyed by repo full_name
    '''
    repo_dict = {}
    
    # check for rate limit
    limit_wait()
    
    try:        
        repo_dict[repo.full_name] = [f.login for f in repo.get_contributors()]
    except:
        repo_dict[repo.full_name] = ''
        
    return repo_dict

In [551]:
_ = db.drop_collection('git_users_following')
_ = db.create_collection('git_users_following')

In [553]:
#db.collection_names()

In [554]:
user_following_upsert(user)

In [555]:
db.git_users_following.count()

1

In [556]:
for i in db.git_users_following.find():
    pprint(i)

{'_id': 98668, 'login': 'ptwobrussell', 'user_following': []}


In [359]:
def repo_metadata(repo):
    repo_keys = ['id', 'owner', 'name', 'full_name', 'description', 'fork', 'html_url', 'homepage', 
             'language', 'forks_count', 'size', 'open_issues_count', 'has_issues', 'has_wiki', 'has_downloads', 
             'pushed_at', 'created_at', 'subscribers_count', 'stargazers_count']
    
    owner_keys = ['type', 'login', 'id', 'site_admin']
    
    # check for rate limit
    limit_wait()
    
    # get returned payload raw data into dictionary
    raw_dict = dict(repo.raw_data)
    
    # trim repo by keys listed above
    repo_dict = {k: raw_dict[k] for k in repo_keys}
    
    # trim owner by keys listed above
    owner_dict = {k: raw_dict['owner'][k] for k in owner_keys}
    repo_dict['owner'] = owner_dict
    
    return repo_dict

# User Metadata Functions

In [542]:
def user_following(user):
    '''
    IN: github user object
    RETURN: dictionary of users (login name) followed by user (passed) 
            keyed by passed user's login
    '''
    user_dict = {}
    
    # check for rate limit
    limit_wait()

    try:
        user_dict['_id'] = user.id
        user_dict['login'] = user.login
        user_dict['user_following'] = [f.name for f in user.get_following()]
    except:
        user_dict['_id'] = user.id
        user_dict['login'] = user.login
        user_dict['user_following'] = []
        
    return user_dict

In [552]:
def user_following_upsert(user):
    repo_meta = repo_metadata(repo)
    col = db['git_users_following']
    
    try:
        col.update_one(
            {'_id': user.id}, 
            {'$set': {col.insert_one(user_following(user))}
            }, upsert=True)
    except:
        pass
    return None

# User Followers

In [520]:
def user_followers(user):
    '''
    IN: github user object
    RETURN: dictionary of users (login name) who follow user (passed)
            keyed by passed user's login
    '''
    user_dict = {}
    
    # check for rate limit
    limit_wait()
    
    try:
        user_dict['_id'] = user.id
        user_dict['login'] = user.login
        user_dict['user_followers'] = [f.login for f in user.get_followers()]
    except:
        user_dict['_id'] = user.id
        user_dict['login'] = user.login
        user_dict['user_followers'] = []
    
    return user_dict

In [527]:
def user_followers_upsert(user):
    repo_meta = repo_metadata(repo)
    col = db['git_users_followers']
    
    try:
        col.update_one(
            {'_id': user.id}, 
            {'$set': {col.insert_one(user_followers(user))}
            }, upsert=True)
    except:
        pass
    return None

# Users Stargazers

In [506]:
def user_starred(user):
    '''
    IN: github user object
    RETURN: dictionary of repos (full_name) starred by user (passed)
            keyed by passed user's login
    '''
    user_dict = {}
    
    # check for rate limit
    limit_wait()
    
    try:
        user_dict['_id'] = user.id
        user_dict['login'] = user.login
        user_dict['starred_repos'] = [f.full_name for f in user.get_starred()]
    except:
        user_dict['_id'] = user.id
        user_dict['login'] = user.login
        user_dict['starred_repos'] = ''
        
    return user_dict

In [509]:
def user_starred_upsert(user):
    repo_meta = repo_metadata(repo)
    col = db['git_users_starred']
    
    try:
        col.update_one(
            {'_id': user.id}, 
            {'$set': {col.insert_one(user_starred(user))}
            }, upsert=True)
    except:
        pass
    return None

# User Metadata

In [20]:
def user_metadata(user):
    user_keys = ['email', 'followers', 'hireable', 'login', 'id', 'bio', 'avatar_url', 'company', 
                 'updated_at', 'type', 'created_at', 'name', 'location', 'html_url', 'public_repos', 
                 'blog', 'public_gists', 'following']
    # check for rate limit
    limit_wait()
        
    # get returned payload raw data into dictionary
    raw_dict = dict(user.raw_data)
    
    # trim user by keys listed above
    user_dict = {k: raw_dict[k] for k in user_keys}
    
    return user_dict

In [160]:
def repo_meta_upsert(repo):
    repo_meta = repo_metadata(repo)
    col = db['github_repos_meta']
    
    try:
        col.update_one(
            {'full_name': repo.full_name}, 
            {'$set':{col.insert_one(repo_meta)}}, 
            upsert=True)
    except:
        pass
    return None

# Repo Files

In [329]:
def repo_scripts_upsert(repo, file_name, raw_file_cont, file_ext, doc_name):
    '''
    '''
    # strip newline and tab chars from raw file content
    col = db['github_repos_docs']
    doc_name = '{0}_{1}'.format(repo.id, doc_name)
    strip_file_cont = raw_file_cont.replace('\n', ' ')
    strip_file_cont = strip_file_cont.replace('\t', ' ')

    try:
        col.update_one(
            {'document' : doc_name}, 
            {'$set':{
                    'file_name': file_name,
                    #'raw_file': raw_file_cont,
                    #'strip_file': strip_file_cont,
                    'extension': file_ext,
                    'full_name' : repo.full_name,
                    'document' : doc_name}
                }, 
            upsert=True)
    except:
        pass
    return None

In [330]:
def upload_files(repo, doc_ct, _path='.'):
    '''
    IN: github repo object and path to folder
    RETRUN: all files from repo except those in the exclusion list
    '''
    # only files with these extensions
    extensions = ['py', 'md']

    # check for rate limit
    limit_wait()
    
    # grab all contents in the main directory
    dir_contents = repo.get_dir_contents(_path)
    repo_fullname = repo.full_name

    for content in dir_contents:
        # if item is a directory then recursively navigate lower to get files inside
        if content.type == 'dir':
            upload_files(repo, doc_ct, _path=content.path)

        else:
            doc_ct += 1
            # get file extension
            file_ext = content.name.split('.')[-1]
            #file_ext = file_split[-1]

            if file_ext in extensions:
                # try to decode, but return blank if fail
                try:
                    raw_file_content = content.decoded_content.decode(errors='replace')
                except:
                    raw_file_content = ''
                    
                # actual file name, mongodb doc stored name
                file_name = content.name
                doc_name = 'doc_{0}'.format(doc_ct)
                
                # add to collection if under limit (>16mb)
                file_size = sys.getsizeof(raw_file_content)
                if file_size < 16000000:
                    repo_scripts_upsert(repo, file_name, raw_file_content, file_ext, doc_name)
                else:
                    print('FILE SIZE LIMIT: {0} -- {1}'.format(file_name, file_size))
    return None

# Wait function

In [337]:
def limit_wait():
    # wait till rate limit is refreshed
    rate, _ = get_remaining('core')
    while (rate < 10):
        time.sleep(300)

# Search Github for 'Python' repositories

In [ ]:
for repo in git_client.search_repositories('python')[5:6]:
    doc_ct = 0
    
    # ensure its a python tagged dirctory
    if repo.language.lower() == 'python':
        
        #############################
        # INSERT FURTHER CALLS HERE #
        
        print('Working on {0}'.format(repo.full_name))
        upload_files(repo, doc_ct)

        # insert metadata
        repo_meta_upsert(repo) # 9.2
        
        #############################
        
    # don't get cut off
    time.sleep(randint(2,9))
    
print('DONE')

In [ ]:
for name in db.collection_names():
    print('Number of documents in \"{0}\" collection: {1}'.format(name, db[name].count()))

In [ ]:
#for cursor in db['github_repos'].find():
#    pprint(cursor)